# 차원 축소 (Dimensionality Reduction)



## 1. 개요

### 배경
현실 데이터는 수십, 수백 개의 특성을 가진 **고차원 데이터**인 경우가 많습니다.  
차원이 높아지면 → 시각화가 어려워지고, 계산량이 폭발하며, 모델이 과적합되는 문제가 발생합니다.  

👉 **차원 축소(Dimensionality Reduction)** 는  
- 데이터의 중요한 정보를 유지하면서,  
- 불필요하거나 중복된 차원을 줄여  
- 시각화, 처리 효율성, 모델 성능 향상에 기여합니다.  

## 2. 학습 목표
이 토픽을 수강한 뒤, 수강생은 다음을 할 수 있어야 합니다:

- 차원 축소의 필요성과 이점을 설명할 수 있다.  
- PCA와 LDA의 원리를 이해하고 실제 데이터에 적용할 수 있다.  
- 차원 축소를 통해 데이터 시각화 및 모델 성능 평가를 수행할 수 있다.  

## 3. 주요 학습 개념

- **차원 축소란?**
  - 차원 축소의 정의와 필요성  
  - 차원 축소의 이점 (시각화, 처리 효율성, 성능 향상 등)  

- **PCA (Principal Component Analysis)**  
  - PCA의 기본 개념  
  - 공분산 행렬과 고유벡터  
  - 주성분 선택 원리  
  - scikit-learn을 사용한 PCA 구현  

- **LDA (Linear Discriminant Analysis)**  
  - LDA의 기본 개념  
  - 클래스 분리 기준 (between-class variance / within-class variance)  
  - scikit-learn을 사용한 LDA 구현  

- **차원 축소 활용**  
  - 차원 축소를 통한 데이터 시각화 (2D, 3D 표현)  
  - 차원 축소 후 모델 성능 비교  
  - 차원 축소 vs 피처 선택 차이점  

# 0. 들어가기

데이터가 가진 **차원(dimension)** 이란, 하나의 데이터가 가지고 있는 **특성(feature)** 의 개수를 의미합니다.  
예:  
- 키, 몸무게, 나이 → 3차원 데이터  
- 픽셀 784개로 이루어진 MNIST 숫자 이미지 → 784차원 데이터  

👉 차원이 커질수록 나타나는 문제:  
- **시각화 불가능**: 3차원 이상은 인간이 직관적으로 보기 어렵다.  
- **계산량 폭발**: 차원이 늘어나면 거리 계산, 행렬 연산이 급격히 늘어난다.  
- **차원의 저주(Curse of Dimensionality)**: 차원이 커질수록 데이터가 희소해져 모델 학습이 어려워진다.  

따라서 차원을 줄이는 기술, 즉 **차원 축소(Dimensionality Reduction)**가 필요하다.  

# 1. 차원 축소 기본 개념

## 1.1 차원 축소란?
- 원래 데이터의 중요한 정보를 유지하면서 **특성의 개수를 줄이는 과정**  
- 불필요한 변수 제거, 데이터 압축, 시각화에 활용  


## 1.2 차원 축소의 이점
1. **시각화 용이**  
   - 2D, 3D로 축소하여 데이터 패턴을 그림으로 확인 가능  
2. **처리 효율성 향상**  
   - 계산량이 줄어 모델 훈련 속도 증가  
3. **잡음 제거**  
   - 중요하지 않은 특성을 줄여 모델 성능 향상  
4. **과적합 방지**  
   - 복잡성을 줄여 일반화 성능 향상


## 1.3 차원 축소 방법
- **PCA (주성분 분석)**: 분산이 가장 큰 방향을 찾아 차원 축소  
- **LDA (선형 판별 분석)**: 클래스 간 분리를 최대화하는 방향으로 축소  
- (심화) t-SNE, UMAP: 비선형 데이터 시각화에 강력

## 1.4 간단한 예시 (Iris 데이터셋 시각화 전후)

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터 불러오기
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["target"] = iris.target

# 4차원 데이터를 pairplot으로 확인 (복잡함)
sns.pairplot(df, hue="target", vars=iris.feature_names)
plt.show()

# 4차원 데이터는 시각적으로 해석하기 어려움  
# 이후 PCA, LDA 등을 통해 2D로 줄이면 훨씬 직관적으로 볼 수 있음 

### ✅ 체크포인트
- 차원 = 데이터의 특성 개수  
- 차원이 높으면 계산량과 복잡성이 증가 → 차원의 저주 발생  
- 차원 축소는 시각화, 효율성, 성능 향상에 도움을 준다  
- 대표 기법: PCA, LDA (→ 다음 장에서 학습)  

# 2. PCA (Principal Component Analysis)

## 2.1 PCA란?
- **고차원 데이터를 저차원으로 변환**하는 가장 대표적인 기법  
- 데이터의 **분산(variance)**이 가장 큰 방향을 찾아 새로운 축(주성분, Principal Component)을 만든다  
- 이렇게 만든 축을 따라 데이터를 투영하여 차원을 줄인다  

👉 핵심 아이디어:  
"데이터를 가장 잘 설명할 수 있는 방향(축)을 찾아, 그 축만 남기고 나머지는 버리자"

## 2.2 PCA 절차
1. 데이터 표준화 (평균=0, 표준편차=1)  
2. 공분산 행렬 계산  
3. 공분산 행렬의 고유벡터와 고유값 계산  
4. 고유값이 큰 순서대로 주성분 선택  
5. 원래 데이터를 주성분 축으로 변환

## 2.3 PCA 시각적 직관
예를 들어, 2차원 데이터가 대각선 방향으로 퍼져 있다면  
👉 원래 축(x, y축) 대신, **대각선 방향 축**이 데이터의 변동을 더 잘 설명한다.

## 2.4 Scikit-learn으로 PCA 구현

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 불러오기
iris = load_iris()
X = iris.data
y = iris.target

# 1) 데이터 표준화
X_scaled = StandardScaler().fit_transform(X)

# 2) PCA (2차원으로 축소)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 3) DataFrame 변환
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca["target"] = y

# 4) 시각화
plt.figure(figsize=(8,6))
for target in set(y):
    subset = df_pca[df_pca["target"] == target]
    plt.scatter(subset["PC1"], subset["PC2"], label=iris.target_names[target])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.title("PCA on Iris Dataset")
plt.show()

## 2.5 주성분의 분산 설명력
PCA는 각 주성분이 원래 데이터의 분산을 얼마나 설명하는지도 알려준다.  

```python
print("주성분별 설명된 분산 비율:", pca.explained_variance_ratio_)
print("설명된 총 분산 비율:", sum(pca.explained_variance_ratio_))
```

👉 일반적으로, **앞 몇 개의 주성분만으로도 대부분의 정보를 유지**할 수 있다.

### ✅ 체크포인트
- PCA는 데이터의 **분산이 큰 방향**을 찾아 차원을 줄이는 기법이다.  
- 주성분은 공분산 행렬의 고유벡터로 정의된다.  
- Scikit-learn으로 손쉽게 PCA를 적용할 수 있다.  
- `explained_variance_ratio_`를 통해 정보 보존 정도를 확인할 수 있다.  


# 3. LDA (Linear Discriminant Analysis)

## 3.1 LDA란?
- **지도 학습 기반 차원 축소 기법**  
- PCA는 클래스(label)를 고려하지 않지만, **LDA는 클래스 정보를 사용**  
- 목표: 서로 다른 클래스 간 분리가 잘 되도록 새로운 축을 찾는 것  

👉 직관적으로, "클래스 간의 거리는 최대화, 클래스 내부의 분산은 최소화"  

## 3.2 LDA 절차
1. 클래스별 평균과 전체 평균 계산  
2. 클래스 간 분산(between-class scatter)과 클래스 내 분산(within-class scatter) 계산  
3. 두 분산 비율을 최대화하는 축 찾기  
4. 데이터를 그 축에 투영  

## 3.3 LDA와 PCA 차이점
- PCA: **데이터의 분산**을 가장 잘 설명하는 축 찾기 (비지도 학습)  
- LDA: **클래스 간 분리**를 가장 잘 설명하는 축 찾기 (지도 학습)  


## 3.4 Scikit-learn으로 LDA 구현

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.datasets import load_iris
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 불러오기
iris = load_iris()
X = iris.data
y = iris.target

# LDA (2차원으로 축소)
lda = LDA(n_components=2)
X_lda = lda.fit_transform(X, y)

# DataFrame 변환
df_lda = pd.DataFrame(X_lda, columns=["LD1", "LD2"])
df_lda["target"] = y

# 시각화
plt.figure(figsize=(8,6))
for target in set(y):
    subset = df_lda[df_lda["target"] == target]
    plt.scatter(subset["LD1"], subset["LD2"], label=iris.target_names[target])
plt.xlabel("LD1")
plt.ylabel("LD2")
plt.legend()
plt.title("LDA on Iris Dataset")
plt.show()

## 3.5 LDA의 활용
- **시각화**: 고차원 데이터를 2D/3D로 줄여 시각적으로 클래스 분포 확인  
- **분류 성능 향상**: 차원을 줄이면서도 클래스 정보를 반영하여 더 잘 분리된 데이터 생성  
- **전처리 단계**: 분류 모델 적용 전 데이터 축소

### ✅ 체크포인트
- LDA는 클래스 정보를 활용하는 지도학습 기반 차원 축소 기법이다.  
- 클래스 간 분산은 최대화, 클래스 내 분산은 최소화한다.  
- PCA와 달리 LDA는 "클래스 분리"에 초점이 있다.  
- Scikit-learn을 통해 손쉽게 LDA를 적용하고 시각화할 수 있다.  

# 4. 차원 축소 활용

## 4.1 데이터 시각화
차원 축소의 가장 직관적인 효과는 **고차원 데이터를 2D 또는 3D로 시각화**할 수 있다는 점입니다.  

예: Iris 데이터셋을 PCA와 LDA로 축소 후 시각화

In [ ]:
# 이미 PCA, LDA로 변환된 데이터 (X_pca, X_lda)
import matplotlib.pyplot as plt

# PCA 결과 시각화
plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=y, cmap="viridis", edgecolor="k")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA Visualization (Iris)")
plt.colorbar()
plt.show()

# LDA 결과 시각화
plt.figure(figsize=(8,6))
plt.scatter(X_lda[:,0], X_lda[:,1], c=y, cmap="rainbow", edgecolor="k")
plt.xlabel("LD1")
plt.ylabel("LD2")
plt.title("LDA Visualization (Iris)")
plt.colorbar()
plt.show()

👉 결과:  
- PCA는 데이터의 분산을 최대한 보존 → 전체 구조 파악  
- LDA는 클래스 분리를 최적화 → 그룹이 더 명확히 구분

## 4.2 모델 성능 비교
차원 축소 후 **모델을 학습**하여 성능 차이를 비교할 수 있습니다.  

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 분할 (원본 데이터 vs PCA 변환 데이터)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 원본 데이터
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("원본 데이터 정확도:", accuracy_score(y_test, y_pred))

# PCA 데이터
X_pca_train, X_pca_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)
lr.fit(X_pca_train, y_train)
y_pred_pca = lr.predict(X_pca_test)
print("PCA 데이터 정확도:", accuracy_score(y_test, y_pred_pca))

# LDA 데이터
X_lda_train, X_lda_test, y_train, y_test = train_test_split(X_lda, y, test_size=0.2, random_state=42)
lr.fit(X_lda_train, y_train)
y_pred_lda = lr.predict(X_lda_test)
print("LDA 데이터 정확도:", accuracy_score(y_test, y_pred_lda))

👉 보통:  
- PCA → 데이터 압축 후에도 성능 유지 또는 향상 가능  
- LDA → 클래스 분리도를 높여 분류 성능 개선  



## 4.3 피처 선택 vs 차원 축소
- **피처 선택 (Feature Selection)**  
  - 원래의 변수 중 중요한 것만 고름  
  - 해석이 쉽다 (예: “꽃받침 길이와 꽃잎 폭이 중요하다”)  

- **차원 축소 (Dimensionality Reduction)**  
  - 원래의 변수를 조합해 새로운 변수(주성분, 판별축)를 생성  
  - 해석은 어렵지만 더 효율적일 수 있다  

👉 즉, 피처 선택은 "원래 특성 중 일부만 고르기", 차원 축소는 "새로운 특성 만들기"

### ✅ 체크포인트
- 차원 축소는 데이터 시각화, 계산 효율, 성능 향상에 유용하다.  
- PCA는 분산 보존, LDA는 클래스 분리에 초점.  
- 차원 축소 후에도 모델 성능을 반드시 다시 평가해야 한다.  
- 피처 선택과 차원 축소는 다른 접근 방식이므로 구분해야 한다.

### 차원 축소 – 실습 문제

### 문제 1. PCA 변환
Iris 데이터셋을 불러와서,  
`n_components=2`인 PCA를 적용한 뒤 2차원 산점도로 시각화하세요.  

<details>
<summary>정답 보기</summary>

```python
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

iris = load_iris()
X, y = iris.data, iris.target

# 표준화
X_scaled = StandardScaler().fit_transform(X)

# PCA 적용
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 시각화
plt.scatter(X_pca[:,0], X_pca[:,1], c=y, cmap="viridis", edgecolor="k")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA on Iris")
plt.show()
```
</details>



### 문제 2. 주성분 분산 설명력
문제 1에서 학습한 PCA 객체의  
각 주성분별 **설명된 분산 비율**과 **누적 설명된 분산 비율**을 출력하세요.  

<details>
<summary>정답 보기</summary>

```python
print("주성분별 설명된 분산 비율:", pca.explained_variance_ratio_)
print("누적 설명된 분산 비율:", sum(pca.explained_variance_ratio_))
```
</details>

### 문제 3. LDA 변환
Iris 데이터셋에 LDA를 적용해 2차원으로 축소하고,  
클래스별로 다른 색으로 시각화하세요.  

<details>
<summary>정답 보기</summary>

```python
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=2)
X_lda = lda.fit_transform(X, y)

plt.scatter(X_lda[:,0], X_lda[:,1], c=y, cmap="rainbow", edgecolor="k")
plt.xlabel("LD1")
plt.ylabel("LD2")
plt.title("LDA on Iris")
plt.show()
```
</details>

### 문제 4. 차원 축소 후 분류 모델 성능 비교
Iris 데이터셋에서 **로지스틱 회귀** 모델을 적용했을 때,  
- 원본 데이터,  
- PCA 변환 데이터,  
- LDA 변환 데이터  

의 정확도를 비교하세요.  

<details>
<summary>정답 보기</summary>

```python
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)
print("원본 데이터 정확도:", accuracy_score(y_test, lr.predict(X_test)))

# PCA 데이터
X_pca_train, X_pca_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)
lr.fit(X_pca_train, y_train)
print("PCA 데이터 정확도:", accuracy_score(y_test, lr.predict(X_pca_test)))

# LDA 데이터
X_lda_train, X_lda_test, y_train, y_test = train_test_split(X_lda, y, test_size=0.2, random_state=42)
lr.fit(X_lda_train, y_train)
print("LDA 데이터 정확도:", accuracy_score(y_test, lr.predict(X_lda_test)))
```
</details>



### 문제 5. 피처 선택 vs 차원 축소
피처 선택과 차원 축소의 차이를 간단히 설명하세요.  

<details>
<summary>정답 보기</summary>

- **피처 선택 (Feature Selection)**: 원래의 특성 중 중요한 일부만 선택 (예: 변수 중요도 기반 선택). 해석이 쉽다.  
- **차원 축소 (Dimensionality Reduction)**: 원래 특성을 조합해 새로운 특성(주성분, 판별축)을 생성. 해석은 어렵지만 효율적일 수 있다.  
</details>

### ✅ 체크포인트
- PCA는 데이터의 분산을 최대한 보존하는 축으로 투영한다.  
- LDA는 클래스 분리를 최대화하는 축으로 데이터를 투영한다.  
- 차원 축소 후에도 모델 성능을 반드시 재평가해야 한다.  
- 피처 선택과 차원 축소는 서로 다른 접근 방식이다.  
